## Test config on easy, medium and hard and all images

Computes SSIM and FID scores.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys,inspect
sys.path.insert(0,"..")

In [3]:
import time
from collections import OrderedDict
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
import os
import numpy as np
import torch
from torch.autograd import Variable
from tensorboardX import SummaryWriter
import cv2

?


In [4]:
writer = SummaryWriter('runs/G1G2')
SIZE=320
NC=14
def generate_label_plain(inputs):
    size = inputs.size()
    pred_batch = []
    for input in inputs:
        input = input.view(1, NC, 256,192)
        pred = np.squeeze(input.data.max(1)[1].cpu().numpy(), axis=0)
        pred_batch.append(pred)

    pred_batch = np.array(pred_batch)
    pred_batch = torch.from_numpy(pred_batch)
    label_batch = pred_batch.view(size[0], 1, 256,192)

    return label_batch

def generate_label_color(inputs):
    label_batch = []
    for i in range(len(inputs)):
        label_batch.append(util.tensor2label(inputs[i], opt.label_nc))
    label_batch = np.array(label_batch)
    label_batch = label_batch * 2 - 1
    input_label = torch.from_numpy(label_batch)

    return input_label
def complete_compose(img,mask,label):
    label=label.cpu().numpy()
    M_f=label>0
    M_f=M_f.astype(np.int)
    M_f=torch.FloatTensor(M_f).cuda()
    masked_img=img*(1-mask)
    M_c=(1-mask.cuda())*M_f
    M_c=M_c+torch.zeros(img.shape).cuda()##broadcasting
    return masked_img,M_c,M_f

def compose(label,mask,color_mask,edge,color,noise):
    # check=check>0
    # print(check)
    masked_label=label*(1-mask)
    masked_edge=mask*edge
    masked_color_strokes=mask*(1-color_mask)*color
    masked_noise=mask*noise
    return masked_label,masked_edge,masked_color_strokes,masked_noise
def changearm(old_label):
    label=old_label
    arm1=torch.FloatTensor((data['label'].cpu().numpy()==11).astype(np.int))
    arm2=torch.FloatTensor((data['label'].cpu().numpy()==13).astype(np.int))
    noise=torch.FloatTensor((data['label'].cpu().numpy()==7).astype(np.int))
    label=label*(1-arm1)+arm1*4
    label=label*(1-arm2)+arm2*4
    label=label*(1-noise)+noise*4
    return label

## Select configuration

In [5]:
## Load models
exp_name = "resunet_g1"
model_dir = "../../train_src/checkpoints/"


opt = TrainOptions().parse()
opt.load_pretrain = model_dir+exp_name

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
datapairs: test_pairs.txt
dataroot: ../datasets/acgpn_data/try_on_testing/
debug: False
display_freq: 100
display_winsize: 512
fineSize: 512
gpu_ids: [0]
input_nc: 3
isTrain: True
label_nc: 20
lambda_feat: 10.0
loadSize: 512
load_pretrain: ./checkpoints/label2city
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 4
n_blocks_local: 3
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: label2city
ndf: 64
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: test
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_dropout: False
verbose: False
which_epoch: latest
-------------- End ------------

In [47]:
opt.datapairs = "medium_same.txt"

In [48]:
category = "medium"

In [49]:
# Path to tryon
os.makedirs('../tryons_category/{}/{}'.format(exp_name, category),exist_ok=True)

In [50]:
# Select test data
opt.dataroot = "../../datasets/acgpn_data/try_on_testing/"

iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')
# Check models used
print(opt.load_pretrain)

../../train_src/checkpoints/resunet_g1


In [51]:
if opt.continue_train:
    try:
        start_epoch, epoch_iter = np.loadtxt(iter_path , delimiter=',', dtype=int)
    except:
        start_epoch, epoch_iter = 1, 0
    print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))        
else:    
    start_epoch, epoch_iter = 1, 0

In [52]:
if opt.debug:
    opt.display_freq = 1
    opt.print_freq = 1
    opt.niter = 1
    opt.niter_decay = 0
    opt.max_dataset_size = 10   

In [53]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('# Inference images = %d' % dataset_size)

CustomDatasetDataLoader
dataset [AlignedDataset] was created
../../datasets/acgpn_data/try_on_testing/test_label label
../../datasets/acgpn_data/try_on_testing/test_label label
../../datasets/acgpn_data/try_on_testing/test_img img
../../datasets/acgpn_data/try_on_testing/test_img img
../../datasets/acgpn_data/try_on_testing/test_edge edge
../../datasets/acgpn_data/try_on_testing/test_edge edge
../../datasets/acgpn_data/try_on_testing/test_mask mask
../../datasets/acgpn_data/try_on_testing/test_mask mask
../../datasets/acgpn_data/try_on_testing/test_colormask colormask
../../datasets/acgpn_data/try_on_testing/test_colormask colormask
../../datasets/acgpn_data/try_on_testing/test_color color
../../datasets/acgpn_data/try_on_testing/test_color color
# Inference images = 514


### Load model

In [54]:
model = create_model(opt)

latest_net_U.pth
latest_net_G1.pth
latest_net_G2.pth
latest_net_G.pth


In [55]:
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

In [56]:
data_dummy = next(iter(dataset))

/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode ins

In [57]:
data_dummy.keys()

dict_keys(['label', 'label_ref', 'image', 'image_ref', 'path', 'path_ref', 'edge', 'color', 'mask', 'colormask', 'pose', 'name'])

In [58]:
data_dummy['path'], data_dummy['path_ref'], data_dummy['name']

(['../../datasets/acgpn_data/try_on_testing/test_label/005533_0.png'],
 ['../../datasets/acgpn_data/try_on_testing/test_label/012137_0.png'],
 ['005533_0.jpg'])

In [59]:
data_dummy['mask'].shape

torch.Size([1, 1, 256, 192])

In [60]:
data_dummy['image'].shape

torch.Size([1, 3, 256, 192])

In [61]:
# import cv2
# from PIL import Image, ImageDraw
# import matplotlib.pyplot as plt

In [62]:
# A = cv2.imread(data_dummy['path'][0], -1)
# B = cv2.imread(data_dummy['path_ref'][0], -1)

In [63]:
# img_stack = np.hstack((A, B))
# plt.imshow(img_stack)
# plt.show()

In [64]:
start_epoch, opt.niter + opt.niter_decay + 1

(1, 201)

In [65]:
import pytorch_ssim
import torch
from torch.autograd import Variable

In [66]:
step = 0 
ssims = []

for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    #import ipdb; ipdb.set_trace()
    
    epoch_start_time = time.time()
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
        
    # iterate over the dataset
    for i, data in enumerate(dataset, start=epoch_iter):

        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        #save_fake = total_steps % opt.display_freq == display_delta
        save_fake = True

        ##add gaussian noise channel
        ## wash the label
        t_mask = torch.FloatTensor((data['label'].cpu().numpy() == 7).astype(np.float))
        #
        # data['label'] = data['label'] * (1 - t_mask) + t_mask * 4
        mask_clothes = torch.FloatTensor((data['label'].cpu().numpy() == 4).astype(np.int))
        mask_fore = torch.FloatTensor((data['label'].cpu().numpy() > 0).astype(np.int))
        img = data['image'].float().cuda()
        img_fore = data['image'] * mask_fore
        img_fore_wc = img_fore * mask_fore
        all_clothes_label = changearm(data['label'])

        ############## Forward Pass ######################
        losses, fake_image, real_image, input_label,L1_loss,style_loss,clothes_mask,CE_loss,rgb,alpha= model(Variable(data['label'].cuda()),
                                                                                                             Variable(data['edge'].cuda()),
                                                                                                             Variable(img_fore.cuda()),
                                                                                                             Variable(mask_clothes.cuda()),
                                                                                                             Variable(data['color'].cuda()),
                                                                                                             Variable(all_clothes_label.cuda()),
                                                                                                             Variable(data['image'].cuda()),
                                                                                                             Variable(data['pose'].cuda()) ,
                                                                                                             Variable(data['image'].cuda()) ,
                                                                                                             Variable(mask_fore.cuda()))
                                            
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
        loss_G = loss_dict['G_GAN']+torch.mean(CE_loss)#loss_dict.get('G_GAN_Feat',0)+torch.mean(L1_loss)+loss_dict.get('G_VGG',0)

        writer.add_scalar('loss_d', loss_D, step)
        writer.add_scalar('loss_g', loss_G, step)
        writer.add_scalar('loss_CE', torch.mean(CE_loss), step)
        writer.add_scalar('loss_g_gan', loss_dict['G_GAN'], step)

        ############## Display results and errors ##########

        
        ### display output images
        a = generate_label_color(generate_label_plain(input_label)).float().cuda()
        b = real_image.float().cuda()
        c = fake_image.float().cuda()
        d=torch.cat([clothes_mask,clothes_mask,clothes_mask],1)
        combine = torch.cat([a[0],d[0],b[0],c[0],rgb[0]], 2).squeeze()
        # combine=c[0].squeeze()
        cv_img=(combine.permute(1,2,0).detach().cpu().numpy()+1)/2
        if step % 1 == 0:
            writer.add_image('combine', (combine.data + 1) / 2.0, step)
            rgb=(cv_img*255).astype(np.uint8)
            bgr=cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR)
            n=str(step)+'.jpg'
            
            #cv2.imwrite('../samples/'+exp_name+'/'+data['name'][0],bgr)
            
            #########################################
            #import ipdb; ipdb.set_trace()
            
            # Save tryon images
            fake_i =(fake_image[0].permute(1,2,0).detach().cpu().numpy()+1)/2
            fake_i=(fake_i*255).astype(np.uint8)
            fake_i=cv2.cvtColor(fake_i,cv2.COLOR_RGB2BGR)
            
            cv2.imwrite('../tryons_category/'+exp_name+'/'+category+'/'+data['name'][0],fake_i)
            #########################################
            
        step += 1
        #print(step)
        
        
        #import ipdb; ipdb.set_trace()
        
        ##################################################
        # Compute SSIM scores
        def norm(tensor_list):
            mins = tensor_list.min()
            maxs = tensor_list.max()
            normalized_data = (tensor_list - mins) / (maxs - mins)
            return normalized_data
        
        # Normalize image to [0-1]
        real_norm = norm(img)
        fake_norm = norm(fake_image)

        score = pytorch_ssim.ssim(real_norm, fake_norm)
        score = score.item()
        #print("SSIM for {}-----:".format(step), score)
        ssims.append(score)
        ##################################################
        
        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            # print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            # model.module.save('latest')
            # np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')
            pass
        if epoch_iter >= dataset_size:
            break

        # Debug
#         if i == 10:
#             print("Exiting loop.")
#             #i = 0
#             break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
    break

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        # np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()

/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/home/hz/anaconda3/envs/comp6321/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode ins

End of epoch 1 / 200 	 Time Taken: 48 sec


# Compute SSIM score

In [67]:
score = np.array(ssims).mean()
print("SSIM score: {:.4f}".format(score))

SSIM score: 0.8254


In [68]:
#score > 0.8273

# Compute FID scores
(ref: -4.0183)

In [28]:
#!python -m pytorch_fid '../tryons_category/resunet_g1/all_same' '../../datasets/acgpn_data/try_on_testing/test_img'

| Model  | Test all SSIM (higher) | Test easy SSIM (higher) | Test medium SSIM (higher)| Test hard SSIM (higher)| Test FID (lower) |
| ------------- | ------------- | ------------- | ------------- | ------------- |------------- |
| ACGPN Paper  | 0.8669 | - |- |- |14.107 |
| ACGPN All (reimplemented acgpn)  | 0.8727 | - |- |- |12.868 |
| ACGPN 5000  | 0.8231 | 0.8311 |0.8214 |0.8044 |26.167 |
| ResUnet 32filt, BN (resunet_g1, ngf=32) | **0.8273** | **0.8356** | **0.8254** | **0.8083** | **23.032** |